In [1]:
# pip install -r requirements.txt --quiet

In [2]:
import pandas as pd
import data.building_ingestion_utils as biu
# import modeling_utils as mu
import data.data_utils as du
# import optimization_utils as ou
# import weather_ingestion_utils as wiu
import data.energy_price_utils as epu
import json

directory = 'data/site_data/industrial_sites'

dataframes = du.restructure_dataframes(biu.get_dataframes(directory, ';'))

In [3]:
print(f"Total dataframes created: {len(dataframes)}")

for name, df in dataframes.items():
    print(f'DataFrame: {name} loaded with shape: {df.shape}')

Total dataframes created: 3
DataFrame: industrial_site2 loaded with shape: (42760, 29)
DataFrame: industrial_site3 loaded with shape: (42760, 44)
DataFrame: industrial_site1 loaded with shape: (42760, 54)


Data is saved in dictionary of dataframes called `dataframe`

In [4]:
#   new_entry_key = 'energy'
#   new_entry_value = 'kWh'
# add_entry_to_units_dict(file_path, new_entry_key, new_entry_value, verbose=False)

units_dict = biu.read_units_dict_from_json('data/unit_dict.json')

dataframes = biu.add_units_to_column_names(dataframes, units_dict)

du.column_presence_checker(dataframes)

,DataFrame Name,01 General Transformer 1234_Electric_Active Energy (kWh),01 General Transformer 1234_Electric_Active Power (kW),01 General Transformer 1234_Electric_Power Factor (real),01 General_Electric_Active Energy (kWh),01 General_Electric_Active Power (kW),01 General_Electric_Power Factor (real),01 Normal General_Electric_Active Energy (kWh),01 Normal General_Electric_Active Power (kW),01 Normal General_Electric_Power Factor (real),...,General_Technical_Efficiency (%),General_Technical_Flow Rate (m3/h),General_Technical_Pressure (bar),General_Technical_Temperature (C),General_Vapour_Flow Rate (m3/h),General_Vapour_Fumes Temperature (C),General_Vapour_Pressure (bar),General_Vapour_Quantity (m3),General_Water_Flow Rate (m3/h),Time
0,industrial_site2,Column not present,Column not present,Column not present,float64,float64,float64,Column not present,Column not present,Column not present,...,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,object
1,industrial_site3,float64,float64,float64,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,...,float64,float64,float64,float64,Column not present,Column not present,Column not present,Column not present,Column not present,object
2,industrial_site1,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,float64,float64,float64,...,Column not present,Column not present,Column not present,Column not present,float64,float64,float64,float64,float64,object


In [5]:
datetime_format = '%d/%m/%y %H:%M'
dataframes = du.convert_and_transform_date_columns(dict_of_dfs=dataframes, datetime_format=datetime_format, date_column='Time', verbose=True)
du.column_presence_checker(dataframes)



DataFrame: industrial_site2 - Converted Time to datetime
DataFrame: industrial_site3 - Converted Time to datetime
DataFrame: industrial_site1 - Converted Time to datetime


,DataFrame Name,01 General Transformer 1234_Electric_Active Energy (kWh),01 General Transformer 1234_Electric_Active Power (kW),01 General Transformer 1234_Electric_Power Factor (real),01 General_Electric_Active Energy (kWh),01 General_Electric_Active Power (kW),01 General_Electric_Power Factor (real),01 Normal General_Electric_Active Energy (kWh),01 Normal General_Electric_Active Power (kW),01 Normal General_Electric_Power Factor (real),...,Time,day_of_month_cos,day_of_month_sin,day_of_week_cos,day_of_week_sin,hour_cos,hour_sin,is_weekend,month_cos,month_sin
0,industrial_site2,Column not present,Column not present,Column not present,float64,float64,float64,Column not present,Column not present,Column not present,...,datetime64[ns],float64,float64,float64,float64,float64,float64,int64,float64,float64
1,industrial_site3,float64,float64,float64,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,...,datetime64[ns],float64,float64,float64,float64,float64,float64,int64,float64,float64
2,industrial_site1,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,float64,float64,float64,...,datetime64[ns],float64,float64,float64,float64,float64,float64,int64,float64,float64


# Create Target Column

## Create Total Consumption (kW) Column

# Data Cleaning

## Imputing missing values with column averages by each dataframe

In [6]:
dataframes = du.impute_missing_values(dataframes)

In [7]:
du.nan_checker(dataframes)

,DataFrame Name,01 General Transformer 1234_Electric_Active Energy (kWh),01 General Transformer 1234_Electric_Active Power (kW),01 General Transformer 1234_Electric_Power Factor (real),01 General_Electric_Active Energy (kWh),01 General_Electric_Active Power (kW),01 General_Electric_Power Factor (real),01 Normal General_Electric_Active Energy (kWh),01 Normal General_Electric_Active Power (kW),01 Normal General_Electric_Power Factor (real),...,Time,day_of_month_cos,day_of_month_sin,day_of_week_cos,day_of_week_sin,hour_cos,hour_sin,is_weekend,month_cos,month_sin
0,No NaNs,Column not present,Column not present,Column not present,No NaNs,No NaNs,No NaNs,Column not present,Column not present,Column not present,...,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs
1,No NaNs,No NaNs,No NaNs,No NaNs,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,...,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs
2,No NaNs,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,No NaNs,No NaNs,No NaNs,...,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs


# Create Total Consumption (kWh) Column

In [8]:
# du.check_column_names(dataframes)
tempdf = dataframes['industrial_site1'].filter(['01 Normal General_Electric_Active Energy (kWh)','General_Electric_Active Energy (kWh)'], axis=1)
tempdf.head()


,01 Normal General_Electric_Active Energy (kWh),General_Electric_Active Energy (kWh)
0,441.0,123.0
1,412.0,117.0
2,440.0,123.0
3,448.0,126.0
4,410.0,123.0


In [9]:
du.drop_columns_with_keywords(dataframes, ['General_Electric_Active Energy', 'General Transformer 1234', 'Active Energy'])

In [10]:
dataframes = du.calculate_active_energy(dataframes)
du.check_column_names(dataframes)

DataFrame: industrial_site2
Columns: ['DataFrame Name', 'Time', '01 General_Electric_Power Factor (real)', '02 Production_Electric_Power Factor (real)', '03 Chiller Group_Electric_Power Factor (real)', '04 UTA_Electric_Power Factor (real)', '05 Compressors_Electric_Power Factor (real)', '06 Offices_Electric_Power Factor (real)', '07 Data Center_Electric_Power Factor (real)', '08 Technological Centers_Electric_Power Factor (real)', 'General_Electric_Power Factor (real)', '01 General_Electric_Active Power (kW)', '02 Production_Electric_Active Power (kW)', '03 Chiller Group_Electric_Active Power (kW)', '04 UTA_Electric_Active Power (kW)', '05 Compressors_Electric_Active Power (kW)', '06 Offices_Electric_Active Power (kW)', '07 Data Center_Electric_Active Power (kW)', '08 Technological Centers_Electric_Active Power (kW)', 'General_Electric_Active Power (kW)', 'is_weekend', 'hour_sin', 'hour_cos', 'month_sin', 'month_cos', 'day_of_week_sin', 'day_of_week_cos', 'day_of_month_sin', 'day_of_mo

In [11]:
dataframes = du.add_sum_column(dataframes, 'kWh', exclude_cols=['01 Normal General_Electric_Active Energy (kWh)','01 General Transformer 1234_Electric_Active Energy (kWh)', '01 General_Electric_Active Energy (kWh)', 'General_Electric_Active Energy (kWh)'])

In [12]:
du.preview_dict_of_dfs(dataframes)

Preview of DataFrame for industrial_site2:


,DataFrame Name,Time,01 General_Electric_Power Factor (real),02 Production_Electric_Power Factor (real),03 Chiller Group_Electric_Power Factor (real),04 UTA_Electric_Power Factor (real),05 Compressors_Electric_Power Factor (real),06 Offices_Electric_Power Factor (real),07 Data Center_Electric_Power Factor (real),08 Technological Centers_Electric_Power Factor (real),...,01 General_Electric_Active Energy (kWh),02 Production_Electric_Active Energy (kWh),03 Chiller Group_Electric_Active Energy (kWh),04 UTA_Electric_Active Energy (kWh),05 Compressors_Electric_Active Energy (kWh),06 Offices_Electric_Active Energy (kWh),07 Data Center_Electric_Active Energy (kWh),08 Technological Centers_Electric_Active Energy (kWh),General_Electric_Active Energy (kWh),sum_of_kWh
0,industrial_site2,2022-07-01 00:00:00,0.938,0.49300,0.775,0.753,0.953,0.978,-0.77,0.005,...,118.65700,8.097525,40.3000,7.605300,3.978775,0.71883,-0.050050,0.043375,-87828.300,60.693755
1,industrial_site2,2022-07-01 00:15:00,0.947,-0.00480,0.787,0.757,0.951,0.978,-0.80,-0.910,...,133.76375,-0.070680,40.5305,7.664625,3.685125,0.73839,-0.052000,-6.938750,-88643.100,45.557210
2,industrial_site2,2022-07-01 00:30:00,0.928,-0.00400,0.780,0.759,0.951,-0.880,-0.77,-0.910,...,110.66400,-0.061800,39.7800,7.665900,3.732675,-0.59400,-0.048125,-6.711250,-89107.875,43.763400
3,industrial_site2,2022-07-01 00:45:00,0.956,0.00025,0.793,0.758,0.955,-1.000,-0.77,-0.920,...,116.87100,0.003394,32.5130,7.674750,4.034875,-0.70500,-0.050050,-6.624000,-86330.850,36.846969
4,industrial_site2,2022-07-01 01:00:00,0.940,-0.00200,0.781,0.759,0.952,-0.900,-0.78,-0.910,...,120.32000,-0.027250,40.2215,7.684875,4.307800,-0.72900,-0.050700,-6.938750,-86446.575,44.468475




Preview of DataFrame for industrial_site3:


,DataFrame Name,Time,General_Technical_Efficiency (%),02 Chiller Group_Electric_Power Factor (real),03 Aspirator_Electric_Power Factor (real),04 Compressed Air_Electric_Power Factor (real),05 Weaving_Electric_Power Factor (real),06 Ironing_Electric_Power Factor (real),07 UPS_Electric_Power Factor (real),08 General Services_Electric_Power Factor (real),...,04 Compressed Air_Electric_Active Energy (kWh),05 Weaving_Electric_Active Energy (kWh),06 Ironing_Electric_Active Energy (kWh),07 UPS_Electric_Active Energy (kWh),08 General Services_Electric_Active Energy (kWh),09 UTA_Electric_Active Energy (kWh),10 Warehouses_Electric_Active Energy (kWh),11 Winding_Electric_Active Energy (kWh),12 Others_Electric_Active Energy (kWh),sum_of_kWh
0,industrial_site3,2022-07-01 00:00:00,0.0000,1.25,0.625,0.923,1.11,1.070,1.57,0.999,...,132.45050,111.2775,58.8500,0.777150,8.416575,24.00300,2.09250,0.359370,43464302.00,4.346481e+07
1,industrial_site3,2022-07-01 00:15:00,4.2200,1.25,0.623,0.921,1.11,1.050,1.59,0.999,...,120.42075,111.0000,55.1250,0.787050,7.992000,24.38100,2.08290,0.267270,42138614.00,4.213910e+07
2,industrial_site3,2022-07-01 00:30:00,1.9000,1.25,0.622,0.939,1.10,0.946,1.57,0.997,...,122.07000,111.1000,40.4415,0.781075,8.250175,24.41600,2.08290,0.259000,42672303.50,4.267277e+07
3,industrial_site3,2022-07-01 00:45:00,0.0000,1.25,0.623,0.942,1.11,1.110,1.59,1.010,...,121.98900,107.6700,47.1750,0.779100,7.322500,24.16175,2.10145,1.123500,42231791.25,4.223228e+07
4,industrial_site3,2022-07-01 01:00:00,0.0842,1.24,0.621,0.912,1.10,1.110,1.57,1.000,...,118.10400,109.7250,48.2850,0.781075,7.925000,24.66750,2.04315,1.084893,41614152.50,4.161463e+07




Preview of DataFrame for industrial_site1:


,DataFrame Name,Time,General_Natural Gas_Quantity (m3),General_Vapour_Quantity (m3),General_Vapour_Fumes Temperature (C),01 Normal General_Electric_Power Factor (real),02 Chiller_Electric_Power Factor (real),03 Dyeing_Electric_Power Factor (real),04 Ironing_Electric_Power Factor (real),05 Purifier_Electric_Power Factor (real),...,08 Offices Changing Rooms_Electric_Active Energy (kWh),09 Compressed Air_Electric_Active Energy (kWh),10 General Services_Electric_Active Energy (kWh),11 UPS_Electric_Active Energy (kWh),12 UTA_Electric_Active Energy (kWh),14 Laboratory_Electric_Active Energy (kWh),15 Warehouses_Electric_Active Energy (kWh),17 Print_Electric_Active Energy (kWh),General_Electric_Active Energy (kWh),sum_of_kWh
0,industrial_site1,2022-07-01 00:00:00,0.0,0.0,179.0,0.950,0.872,0.907,1.13,0.930,...,16.652650,40.73125,9.68820,2.037400,58.905,0.47515,3.3915,12.422025,110.4495,345.666925
1,industrial_site1,2022-07-01 00:15:00,0.0,0.0,186.0,0.951,0.370,0.905,1.14,0.925,...,16.933750,35.41200,9.28330,2.069100,58.080,0.28764,3.3915,20.405000,113.1600,310.224915
2,industrial_site1,2022-07-01 00:30:00,0.0,0.0,187.0,0.944,0.370,0.914,1.14,0.703,...,16.254975,40.90625,9.25925,2.038850,75.480,0.27922,3.3915,20.046000,122.2245,358.124670
3,industrial_site1,2022-07-01 00:45:00,0.0,4096.0,187.0,0.942,0.367,0.914,1.15,0.930,...,16.137550,40.77500,9.29295,2.020700,60.255,0.26754,3.3915,21.146000,120.4875,337.371990
4,industrial_site1,2022-07-01 01:00:00,0.0,0.0,176.0,0.937,0.866,0.907,1.10,0.690,...,15.801000,35.37300,9.80870,2.072125,56.635,0.43582,3.4170,21.450750,122.2000,341.551145


# Aggregating Data
15 minute data -> hourly data

In [13]:
dataframes = du.aggregate_to_hourly(dataframes, 'Time')
du.preview_dict_of_dfs(dataframes)

Preview of DataFrame for industrial_site2:


,Time,01 General_Electric_Active Energy (kWh),01 General_Electric_Active Power (kW),01 General_Electric_Power Factor (real),02 Production_Electric_Active Energy (kWh),02 Production_Electric_Active Power (kW),02 Production_Electric_Power Factor (real),03 Chiller Group_Electric_Active Energy (kWh),03 Chiller Group_Electric_Active Power (kW),03 Chiller Group_Electric_Power Factor (real),...,DataFrame Name,is_weekend,hour_sin,hour_cos,day_of_month_sin,day_of_month_cos,month_sin,month_cos,day_of_week_sin,day_of_week_cos
Time,,,,,,,,,,,,,,,,,,,,,
0,2022-07-01 00:00:00,479.95575,509.25,0.94225,7.968439,60.175,0.121112,153.12350,195.50,0.78375,...,industrial_site2,0.0,0.000000,1.000000,0.201299,0.97953,-0.5,-0.866025,-0.433884,-0.900969
1,2022-07-01 01:00:00,443.39700,474.00,0.93525,-0.242230,55.475,-0.004325,152.16125,195.25,0.77975,...,industrial_site2,0.0,0.258819,0.965926,0.201299,0.97953,-0.5,-0.866025,-0.433884,-0.900969
2,2022-07-01 02:00:00,425.87675,448.50,0.94950,-0.131350,62.200,-0.002225,128.90925,162.25,0.79450,...,industrial_site2,0.0,0.500000,0.866025,0.201299,0.97953,-0.5,-0.866025,-0.433884,-0.900969
3,2022-07-01 03:00:00,480.08375,503.25,0.95375,-0.084750,56.175,-0.001512,123.00075,157.50,0.78075,...,industrial_site2,0.0,0.707107,0.707107,0.201299,0.97953,-0.5,-0.866025,-0.433884,-0.900969
4,2022-07-01 04:00:00,439.13700,461.00,0.95225,12.848925,57.350,0.248000,124.18700,158.25,0.78475,...,industrial_site2,0.0,0.866025,0.500000,0.201299,0.97953,-0.5,-0.866025,-0.433884,-0.900969




Preview of DataFrame for industrial_site3:


,Time,02 Chiller Group_Electric_Active Energy (kWh),02 Chiller Group_Electric_Active Power (kW),02 Chiller Group_Electric_Power Factor (real),03 Aspirator_Electric_Active Energy (kWh),03 Aspirator_Electric_Active Power (kW),03 Aspirator_Electric_Power Factor (real),04 Compressed Air_Electric_Active Energy (kWh),04 Compressed Air_Electric_Active Power (kW),04 Compressed Air_Electric_Power Factor (real),...,DataFrame Name,is_weekend,hour_sin,hour_cos,day_of_month_sin,day_of_month_cos,month_sin,month_cos,day_of_week_sin,day_of_week_cos
Time,,,,,,,,,,,,,,,,,,,,,
0,2022-07-01 00:00:00,411.2500,329.00,1.25000,260.83000,418.50,0.62325,496.93025,533.75,0.93125,...,industrial_site3,0.0,0.000000,1.000000,0.201299,0.97953,-0.5,-0.866025,-0.433884,-0.900969
1,2022-07-01 01:00:00,424.4125,342.25,1.24000,263.58900,422.75,0.62350,514.90050,555.25,0.92700,...,industrial_site3,0.0,0.258819,0.965926,0.201299,0.97953,-0.5,-0.866025,-0.433884,-0.900969
2,2022-07-01 02:00:00,416.0475,338.25,1.23000,264.94000,424.75,0.62375,523.78225,556.25,0.94175,...,industrial_site3,0.0,0.500000,0.866025,0.201299,0.97953,-0.5,-0.866025,-0.433884,-0.900969
3,2022-07-01 03:00:00,347.1180,337.00,1.02925,259.11625,416.25,0.62250,491.12825,524.50,0.93625,...,industrial_site3,0.0,0.707107,0.707107,0.201299,0.97953,-0.5,-0.866025,-0.433884,-0.900969
4,2022-07-01 04:00:00,416.6500,338.75,1.23000,263.58950,422.75,0.62350,532.80725,565.00,0.94300,...,industrial_site3,0.0,0.866025,0.500000,0.201299,0.97953,-0.5,-0.866025,-0.433884,-0.900969




Preview of DataFrame for industrial_site1:


,Time,01 Normal General_Electric_Active Energy (kWh),01 Normal General_Electric_Active Power (kW),01 Normal General_Electric_Power Factor (real),02 Chiller_Electric_Active Energy (kWh),02 Chiller_Electric_Active Power (kW),02 Chiller_Electric_Power Factor (real),03 Dyeing_Electric_Active Energy (kWh),03 Dyeing_Electric_Active Power (kW),03 Dyeing_Electric_Power Factor (real),...,DataFrame Name,is_weekend,hour_sin,hour_cos,day_of_month_sin,day_of_month_cos,month_sin,month_cos,day_of_week_sin,day_of_week_cos
Time,,,,,,,,,,,,,,,,,,,,,
0,2022-07-01 00:00:00,1691.29300,1786.75,0.94675,117.98575,238.25,0.49475,219.84425,241.50,0.91000,...,industrial_site1,0.0,0.000000,1.000000,0.201299,0.97953,-0.5,-0.866025,-0.433884,-0.900969
1,2022-07-01 01:00:00,1554.18425,1656.25,0.93825,189.32625,218.75,0.86550,220.63825,243.25,0.90700,...,industrial_site1,0.0,0.258819,0.965926,0.201299,0.97953,-0.5,-0.866025,-0.433884,-0.900969
2,2022-07-01 02:00:00,1445.36450,1534.75,0.94175,138.45750,222.75,0.61725,191.79475,214.50,0.89425,...,industrial_site1,0.0,0.500000,0.866025,0.201299,0.97953,-0.5,-0.866025,-0.433884,-0.900969
3,2022-07-01 03:00:00,1596.61300,1690.00,0.94475,161.13000,212.50,0.75675,207.39100,230.75,0.89875,...,industrial_site1,0.0,0.707107,0.707107,0.201299,0.97953,-0.5,-0.866025,-0.433884,-0.900969
4,2022-07-01 04:00:00,1582.32750,1674.75,0.94475,164.73225,217.50,0.75825,218.51250,240.00,0.91050,...,industrial_site1,0.0,0.866025,0.500000,0.201299,0.97953,-0.5,-0.866025,-0.433884,-0.900969


# Ingest Price Data

In [14]:
pd.DataFrame(du.get_min_max_dates(dataframes))

,industrial_site2,industrial_site3,industrial_site1
earliest_date,2022-07-01 00:00:00,2022-07-01 00:00:00,2022-07-01 00:00:00
latest_date,2023-09-19 09:00:00,2023-09-19 09:00:00,2023-09-19 09:00:00


In [15]:
import os
import re
from datetime import datetime, timedelta

# Define the directory containing the files
directory = 'data/energy_price_data/marginalpdbc'

# Regex pattern to match the filenames and extract dates
pattern = re.compile(r'marginalpdbc_(\d{8})\.\d')

# Get a list of files in the directory
files = os.listdir(directory)

# Extract dates from filenames
dates = []
for file in files:
    match = pattern.match(file)
    if match:
        date_str = match.group(1)
        date = datetime.strptime(date_str, '%Y%m%d')
        dates.append(date)

# Sort dates
dates.sort()

# Find the range of dates
if dates:
    start_date = dates[0]
    end_date = dates[-1]

    # Generate the complete date range
    complete_range = set(start_date + timedelta(days=x) for x in range((end_date - start_date).days + 1))

    # Find missing dates
    missing_dates = complete_range - set(dates)

    # Print missing dates
    if missing_dates:
        print("Missing dates:")
        for missing_date in sorted(missing_dates):
            print(missing_date.strftime('%Y%m%d'))
    else:
        print("No missing dates.")
else:
    print("No valid files found in the directory.")

No missing dates.


In [16]:
csv_data = pd.read_csv('data/energy_price_data/spain_daily_market_hourly_prices/marginalpdbc_2022.csv')
csv_data.head()

,Date,Price1,Price2,Year
0,2022-03-18 00:00:00,203.00,203.00,2022
1,2022-03-18 01:00:00,199.00,199.00,2022
2,2022-03-18 02:00:00,198.88,198.88,2022
3,2022-03-18 03:00:00,198.88,198.88,2022
4,2022-03-18 04:00:00,198.88,198.88,2022


In [17]:
dataframes = epu.merge_price_data(dataframes, 'data/energy_price_data/spain_daily_market_hourly_prices/', 'Time', 'Date', 'Price1')


In [18]:
du.add_energy_cost_column(dataframes, 'price_mWh', 'sum_of_kWh')

In [19]:
du.check_column_names(dataframes)

DataFrame: industrial_site2
Columns: ['Time', '01 General_Electric_Active Energy (kWh)', '01 General_Electric_Active Power (kW)', '01 General_Electric_Power Factor (real)', '02 Production_Electric_Active Energy (kWh)', '02 Production_Electric_Active Power (kW)', '02 Production_Electric_Power Factor (real)', '03 Chiller Group_Electric_Active Energy (kWh)', '03 Chiller Group_Electric_Active Power (kW)', '03 Chiller Group_Electric_Power Factor (real)', '04 UTA_Electric_Active Energy (kWh)', '04 UTA_Electric_Active Power (kW)', '04 UTA_Electric_Power Factor (real)', '05 Compressors_Electric_Active Energy (kWh)', '05 Compressors_Electric_Active Power (kW)', '05 Compressors_Electric_Power Factor (real)', '06 Offices_Electric_Active Energy (kWh)', '06 Offices_Electric_Active Power (kW)', '06 Offices_Electric_Power Factor (real)', '07 Data Center_Electric_Active Energy (kWh)', '07 Data Center_Electric_Active Power (kW)', '07 Data Center_Electric_Power Factor (real)', '08 Technological Centers_

# Join Weather Data

In [20]:
from data.weather_data.weather_ingestion_utils import calculate_heat_index

weather_data = pd.read_csv('data/weather_data/processed_data/madrid,spain 2022-07-01 to 2023-09-20.csv')

weather_data['datetime'] = pd.to_datetime(weather_data['datetime'])

weather_data = calculate_heat_index(weather_data)

for name, df in dataframes.items():
    df['datetime_hourly'] = df['Time'].dt.floor('h')
    merged_df = pd.merge(df, weather_data, left_on=df['datetime_hourly'] , right_on=weather_data['datetime'], how='left')
    
    merged_df.drop(columns=['key_0', 'datetime', 'datetime_hourly'], inplace=True)
    
    dataframes[name] = merged_df
    

    

# Save data to CSVs

In [21]:
du.save_dataframes_to_csv(dataframes, 'data/processed_data/industrial_sites_processed', verbose=True)

Dataframe 'industrial_site2' saved to 'data/processed_data/industrial_sites_processed/2024-07-03T01-15-58_industrial_site2_processed.csv'
Dataframe 'industrial_site3' saved to 'data/processed_data/industrial_sites_processed/2024-07-03T01-15-58_industrial_site3_processed.csv'
Dataframe 'industrial_site1' saved to 'data/processed_data/industrial_sites_processed/2024-07-03T01-15-58_industrial_site1_processed.csv'
